In [ ]:
# Importando as bibliotecas
import smtplib
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import pandas as pd

In [ ]:
# Importando a tabela no excel
tabela = '/content/Respostas_a_Receber.xlsx'
df = pd.read_excel(tabela)

In [ ]:
# Definindo o número de envios e intervalo
numero_envios = 3
intervalo_segundos = 10

In [ ]:
# Definindo algumas váriaveis
email_origem = 'viny.rbrito18@gmail.com'
senha_origem = 'dibu oafm wchv admr'
assunto = 'Cotação Pendente'
corpo_email = 'Espero que esta mensagem o encontre bem. Estamos entrando em contato para dar continuidade ao processo de cotação iniciado anteriormente. No entanto, até o momento, ainda não recebemos sua resposta à solicitação de cotação.'

In [ ]:
# Loop para controlar o número de envios
for envio in range(numero_envios):
    for index, row in df.iterrows():
        status, destinatario = row['Status'], row['E-mail']

        if status == 'Aguardando Resposta' and pd.notnull(destinatario):
            msg = MIMEMultipart()
            msg.attach(MIMEText(corpo_email))
            msg['Subject'] = assunto
            msg['From'] = email_origem
            msg['To'] = destinatario

            # Adicionar PDF para envio
            pdf_anexo = '/content/Pedido_para_cotacao.pdf'
            with open(pdf_anexo, 'rb') as file:
                attach = MIMEApplication(file.read(), _subtype="pdf")
                attach.add_header('Content-Disposition', 'attachment', filename='arquivo.pdf')
                msg.attach(attach)

            # Enviar o e-mail
            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()
                server.login(email_origem, senha_origem)
                server.sendmail(email_origem, destinatario, msg.as_string())

            print(f'E-mail com PDF enviado para {destinatario}')

    # Aguardar 10 segundos antes do próximo envio
    if envio < numero_envios - 1:
        print(f'Aguardando 10 segundos antes do próximo envio...')
        time.sleep(intervalo_segundos)

# ...

In [ ]:
# Importando as bibliotecas
import smtplib
import time
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
email_origem = 'viny.rbrito18@gmail.com'
senha_origem = 'dibu oafm wchv admr'
assunto = 'Cotação Pendente'
corpo_email = 'Espero que esta mensagem o encontre bem. Estamos entrando em contato para dar continuidade ao processo de cotação iniciado anteriormente. No entanto, até o momento, ainda não recebemos sua resposta à solicitação de cotação.'

In [ ]:
tabela = '/content/Respostas_a_Receber.xlsx'
df = pd.read_excel(tabela)

In [ ]:
# Adicionando uma coluna para indicar se o fornecedor respondeu ou não
# Considerando que 'Resposta Confirmada' significa 1 (respondeu) e 'Aguardando Resposta' significa 0 (não respondeu)
df['Resposta_Binaria'] = df['Status'].apply(lambda x: 1 if x == 'Resposta Confirmada' else 0)

In [ ]:
# Features que serão usadas para a previsão (por exemplo, Valor em aberto, Data Prevista para Resposta)
features = df[['Valor em aberto', 'Data Prevista para Resposta']]

In [ ]:
# Variável alvo (o que queremos prever)
target = df['Resposta_Binaria']

In [ ]:
# Dividindo o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Treinando o modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Fazendo previsões no conjunto de teste
predictions = model.predict(X_test)

In [ ]:
# Avaliando a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print(f'Acurácia do modelo: {accuracy}')

In [ ]:
# Adicionando previsões ao dataframe original
df['Previsao'] = model.predict(features)

In [ ]:
# Loop para controlar o número de envios
for envio in range(numero_envios):
    for index, row in df.iterrows():
        status, destinatario, previsao = row['Status'], row['E-mail'], row['Previsao']

        # Verificando a previsão do modelo
        if status == 'Aguardando Resposta' and pd.notnull(destinatario) and previsao == 1:
            # Restante do seu código para enviar o e-mail...

            print(f'E-mail com PDF enviado para {destinatario}')

            # Atualizando o status do fornecedor para 'Resposta Confirmada'
            df.at[index, 'Status'] = 'Resposta Confirmada'

    # Atualizando o modelo com novos dados
    model.fit(features, target)

    # Aguardar 10 segundos antes do próximo envio
    if envio < numero_envios - 1:
        print(f'Aguardando 10 segundos antes do próximo envio...')
        time.sleep(intervalo_segundos)